In [209]:
from bokeh.io import show
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper
)
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure
from bokeh.models import HoverTool
from bokeh.plotting import figure, show, output_file, ColumnDataSource
from bokeh.sampledata.us_counties import data as counties
from bokeh.sampledata.unemployment import data as unemployment
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [210]:
import json

In [211]:
with open("nyc.geojson") as f:
    nyc = json.loads(f.read())


In [212]:
# hoods = {neighbourhood: }
d = {}
for i in nyc["features"]:
    key = i["properties"]["neighborhood"]
    xs = []
    ys = []
    coords = {}
    for c in i["geometry"]["coordinates"][0]:
        coords = c
        xs += [c[0]]
        ys += [c[1]]
    
    d[key] = {"xs" : xs, "ys" : ys}
#     print(key,d[key])
#     stop
#     = {"xs" = {i["geometry"]["coordinates"][0]

In [213]:
# d[list(d.keys())[0]]
import sqlite3
import pandas as pd
import numpy as np
db_path = "currencyfairnyc/data/processed/database.db"
conn = sqlite3.connect(db_path)
df = pd.read_sql("select * from taxi_data",con=conn)

In [218]:
df.columns.values

array(['index', 'vendorid', 'pickup_datetime', 'dropoff_datetime',
       'Store_and_fwd_flag', 'rate_code', 'Pickup_longitude',
       'Pickup_latitude', 'Dropoff_longitude', 'Dropoff_latitude',
       'Passenger_count', 'Trip_distance', 'Fare_amount', 'Extra',
       'MTA_tax', 'Tip_amount', 'Tolls_amount', 'Ehail_fee',
       'Improvement_surcharge', 'Total_amount', 'Payment_type',
       'Trip_type', 'Pickup_neighbourhood', 'Dropoff_neighbourhood',
       'Trip_time'], dtype=object)

In [219]:
count_dict = dict(df.groupby("Dropoff_neighbourhood").agg("count")["index"])

In [220]:
print(len(list(d.keys())))

266


In [221]:
good_dic = {}
for hood in list(d.keys()):
    try:
        good_dic[hood] = {}
        good_dic[hood]["count"] = count_dict[hood]
        good_dic[hood]["xs"] = d[hood]["xs"]
        good_dic[hood]["ys"] = d[hood]["ys"]
    except:
        pass
# for item in good_dic:
#     print(len(good_dic[item]))
      
good_dic = {k:v for k,v in good_dic.items() if len(good_dic[k])>1}
# print(len(good_dic))

In [222]:
output_notebook()

# for k,v in good_dic.items():
#     print(v["xs"])
#     stop

hood_xs = [v["xs"] for k,v in good_dic.items()]
hood_ys = [v["ys"] for k,v in good_dic.items()]
hood_counts = [v["count"] for k,v in good_dic.items()]
hood_names = [k for k,v in good_dic.items()]
# hood_colours = 

colors = ["#F1EEF6", "#D4B9DA", "#C994C7", "#DF65B0", "#DD1C77", "#980043"]

hood_colors = [colors[int((len(colors)-1)*np.log1p(count)/max(np.log1p(hood_counts)))] for count in hood_counts]

# for count in hood_counts:
#     print(int(len(colors)*count/max(hood_counts)))
#     print(count/max(hood_counts))
source = ColumnDataSource(data=dict(
    x=hood_xs,
    y=hood_ys,
    color=hood_colors,
#    color="Blue",
    name=hood_names,
    count=hood_counts,
))


Loading BokehJS ...

In [223]:

TOOLS="pan,wheel_zoom,box_zoom,reset,hover,save"

p = figure(title="Texas Unemployment 2009", tools=TOOLS)

p.patches('x', 'y', source=source,
          fill_color='color', fill_alpha=0.7,
          line_color="white", line_width=0.5)

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Name", "@name"),
    ("Counts)", "@count"),
    ("(Long, Lat)", "($x, $y)"),
]

output_file("texas.html", title="texas.py example")

show(p)

Loading BokehJS ...